In [ ]:
import pandas as pd
from rapidfuzz import process, fuzz
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv()
engine = create_engine(os.getenv("DATABASE_URL"))
listings = pd.read_sql("SELECT * FROM filter_listings", engine)
ratings  = pd.read_sql("SELECT residential_complex, rating FROM complex_ratings", engine)
gis_names = ratings["residential_complex"].tolist()
def match(name):
    res = process.extractOne(name, gis_names, scorer=fuzz.token_sort_ratio)
    return res[0] if res and res[1] > 80 else None

listings["matched"] = listings["residential_complex"].apply(match)
merged = listings.merge(
    ratings.rename(columns={"residential_complex": "matched"}),
    on="matched", how="left"
).rename(columns={"rating": "rating_2gis"}).drop(columns="matched")
merged = merged[~(merged == "не указано").any(axis=1)]
merged.to_sql(
    name="joined_listings",
    con=engine,
    if_exists="replace",
    index=False
)

print(f"Wrote {len(merged)} rows to joined_listings in your database")

✅ Wrote 1808 rows to joined_listings in your database
